# Start Ups

**Build a polynomial regression model to predict revenue from 50 start ups data**

From: https://www.udemy.com/machinelearning

In [2]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.graphics.gofplots import qqplot
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn import preprocessing

# Imports other notebooks
import import_ipynb
import regression_lib as rl

importing Jupyter notebook from regression_lib.ipynb


In [2]:
dtf = pd.read_csv('50_Startups.csv')
names = {
    'R&D Spend': 'rd_spend',
    'Administration': 'admin_spend',
    'Marketing Spend': 'marketing_spend',
    'State': 'state',
    'Profit': 'profit'
}
dtf.rename(columns=names, inplace=True)
print(dtf.head())
print(dtf.shape)

    rd_spend  admin_spend  marketing_spend       state     profit
0  165349.20    136897.80        471784.10    New York  192261.83
1  162597.70    151377.59        443898.53  California  191792.06
2  153441.51    101145.55        407934.54     Florida  191050.39
3  144372.41    118671.85        383199.62    New York  182901.99
4  142107.34     91391.77        366168.42     Florida  166187.94
(50, 5)


In [3]:
dir(rl)

['__builtins__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_compute_multiplot_rows',
 'compute_vif',
 'get_dummies',
 'get_ipython',
 'get_rand_rgb_color',
 'np',
 'pd',
 'plt',
 'preprocessing',
 'qqplot',
 'random',
 'shapiro_wilks_test',
 'show_multi_plots',
 'sm',
 'smf',
 'test_H0',
 'variance_inflation_factor']

In [ ]:
''' Functions for generating automatically scatter plots from a dataframe '''

def _compute_multiplot_rows(lst):
    '''
        Computes the rows required to generate a multiplot figure of N rows and 2 columns. 
        lst: list of independent variable columns names in the dataframe (eg. ['xA', 'xB', 'xN'])
    '''
    COL_NUMBER = 2
    
    _lst_len = len(lst)
    _division = int(_lst_len/COL_NUMBER)
    _reminder = _lst_len%COL_NUMBER
    
    if _division == 0:
        return 1
    
    if _reminder == 0:
        return _division
    
    if _reminder > 0:
        return _division + 1
    

def get_rand_rgb_color(col_numb):
    return [np.random.rand(col_numb)]

    
def show_multi_plots(dtf, xy_dct, plt_type):
    '''
    Description
    -----------
        Generates a figure with multiple scatter plots.

    Parameters
    ----------
        dtf: A dataframe containing the independent and dependent variables.
        xy_dct: dictionary with keys x and y, specifying the column names of independent
                and dependent variables respectively.
            Example: { 'x': ['col_xA','col_xB','col_xC'], 'y': 'col_y'}
        plt_type: plot type [scatter | hist]
            
            
    TODO
    ----
        Fix a function fail/error at instruction "axs[r, c].scatter" when passed a list of two colums to plot 
        from dataframe.
    '''
    COL_NUMBER = 2

    x_idx = 0
    calpha = .7
    y_name = xy_dct['y']
    x_lst_len = len(xy_dct['x'])
    rows_number = _compute_multiplot_rows(xy_dct['x'])
    
    fig, axs = plt.subplots(rows_number, 2, figsize=(12, 12))

    for r in range (0, rows_number):
        for c in range (0, COL_NUMBER):
            if x_idx == x_lst_len: 
                break
            x_name = xy_dct['x'][x_idx]
            
            if plt_type == 'scatter':
                axs[r, c].scatter(dtf[x_name], dtf[y_name], c=get_rand_rgb_color(x_lst_len), alpha=calpha)
                axs[r, c].set_ylabel(y_name, fontsize=10)
                axs[r, c].set_xlabel(x_name, fontsize=10)
                
            if plt_type == 'hist':
                axs[r, c].hist(dtf[x_name], color=get_rand_rgb_color(x_lst_len), alpha=calpha)
            
            axs[r, c].set_title(x_name, fontsize=12)
            x_idx += 1
    
    plt.show()

### Plot continous independent variables vs dependent

In [ ]:
xy_dct = { 'x': ['rd_spend','admin_spend','marketing_spend'], 'y': 'profit'}
show_multi_plots(dtf, xy_dct, 'scatter')

print('''rd_spend has a very strong linear relationship to profit, marketing_spend also shows a reasonable correlation, while admin_spend do not seems to have a simple linear relationship to revenue''')

### Plot continous variables vs categorical

In [ ]:
# cols = ['rd_spend','admin_spend']
feats = ['rd_spend','admin_spend','marketing_spend', 'profit']

bxplots = dtf.boxplot(column=feats, by='state', layout=(2,2), figsize=(10,10))

### Are continous variables normal?

In [ ]:
feat = ['rd_spend', 'admin_spend', 'marketing_spend', 'profit']

for f in feat:
    sw_stat, p_value = shapiro_wilks_test(dtf[f])
    print('{} sw_stat={:.3f}, p_value={:.3f}'.format(f, sw_stat, p_value))
    test_H0(p_value)    

In [ ]:
xy_dct = { 'x': ['rd_spend','admin_spend','marketing_spend', 'profit'], 'y': ''}
show_multi_plots(dtf, xy_dct, 'hist')

''' Both the dependent and independent variables adhere to normality. A good start for building a linear model '''

### Exploring collinearity among continous independent variables. 

In [ ]:
vif_dtf = compute_vif(dtf[['rd_spend','admin_spend','marketing_spend']])
print(vif_dtf)

''' VIF results support what was previously observed in the scatter plots, rd_spend and marketing_spend share collinearity. However we will not remove any of them, since we want to test a step-wise model building'''

### Step wise model building with OLS

In [ ]:
significance_level = 0.001 

y = 'profit'

In [ ]:
X = ['rd_spend','admin_spend','marketing_spend']

model_formula_str = "{} ~ {}".format(y, ' + '.join(X))
lm1 = smf.ols(formula=model_formula_str, data=dtf)
results = lm1.fit()

print("Model Formula: ", model_formula_str, "\n")
print(results.summary())

# """
# Accessing model attributes independently 
# List of attributes for statsmodels.regression.linear_model.RegressionResults object
# https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.RegressionResults.html#statsmodels.regression.linear_model.RegressionResults
# print("R^2:", results.rsquared, "\n")
# print("R^2 adj:", results.rsquared_adj, "\n")

# print("Linear coefficients:")
# print(results.params, "\n")

# print('Standard errors (of Linear coefficients): ')
# print(results.bse, "\n")

# print('P-values: ')
# print(results.pvalues, "\n")

# print('Residuals: ')
# print(results.resid[0:3], "... more \n")
# """

In [ ]:
''' Finding the attribute with less contrinution to the model '''
highest_pvalue = max(results.pvalues)
worst_x = results.pvalues[results.pvalues == highest_pvalue]
worst_x_name = worst_x.index[0]
worst_x_pval = worst_x.values[0]
print(worst_x_name, worst_x_pval)

## Backward elimination

sml.ols Regression Results object:  https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.RegressionResults.html#statsmodels.regression.linear_model.RegressionResults

In [ ]:
def backward_elimination(dtf, X_cols, y_col, significance_level):
    
    round = 0 

    while len(X) > 0:
    
        round += 1
        model_formula_str = "{} ~ {}".format(y_col, ' + '.join(X_cols))
        lm1 = smf.ols(formula=model_formula_str, data=dtf)
        results = lm1.fit()
        
        print("\n")
        print("Round", round)
        print("Formula: {}".format(model_formula_str))
        print("R2: {:.4f}, adj.: {:.4f}".format(results.rsquared, results.rsquared_adj))
        print("Fs: {:.4f}, Fpv.: {:.4f}".format(results.fvalue, results.f_pvalue))
        
        ''' Finding the attribute with less contrinution to the model '''
        highest_pvalue = max(results.pvalues)
        
        if highest_pvalue >= significance_level:
            worst_x = results.pvalues[results.pvalues == highest_pvalue]
            worst_x_name = worst_x.index[0]
            worst_x_pval = worst_x.values[0]
            X_cols.remove(worst_x_name)
            print('{} removed from model (Significance pvalue = {:.4f})'.format(worst_x_name, worst_x_pval))
        else:
            return results
    
    return None


X_cols = ['rd_spend','admin_spend','marketing_spend']
y_col = 'profit'

significance_level = 0.001 

ols_results = backward_elimination(dtf, X_cols, y_col, significance_level)

if(ols_results == None):
    print('No significant model built.')
else:
    print(ols_results.summary())

### Visualizing the model

In [ ]:
y = dtf['profit']
X = dtf['rd_spend']
y_pred = ols_results.predict(X)

plt.scatter(X, y, color='r', alpha=.3)
plt.plot(X, y_pred, color='black', alpha=.6)
plt.title('Model plot\n(red: profit, black: profit_pred)')
plt.xlabel('rd_spend')
plt.ylabel('profit')
plt.show()

''' We can eye-ball that for rd_spend < $12K (approx) the model does not work well. We could think of removing those observations from the model'''

## Exploring the residual errors of the model

In [ ]:
y = dtf['profit']
y_pred = ols_results.predict(dtf['rd_spend'])
residuals = y_pred - y

plt.scatter(y, residuals, color='g', alpha=.3)
plt.hlines(0, min(y), max(y), color='black', alpha=.8)
plt.xlabel('profit')
plt.ylabel('residuals')
plt.show()

''' Top left corner, it's not outlier, as there are companies that invested zero in R&D '''

## Adding categorical and other numerical variables to the model
Need to make dummies first

In [ ]:
dtf.head()

In [ ]:
''' Append encoded (dummies) and numerical varibles '''
tmp_dtf = get_dummies(dtf=dtf, categ_cols=['state'])
transf_dtf = pd.concat([dtf, tmp_dtf], axis=1)
transf_dtf.drop(columns=['state'], inplace=True, axis=1)
transf_dtf.columns

In [ ]:
''' Compute model '''
X_cols =  ['rd_spend', 'admin_spend', 'marketing_spend', 'state_California', 'state_Florida']
y_col = 'profit'
formula_str = "{} ~ {}".format(y_col, ' + '.join(X_cols))
lm1 = smf.ols(formula=formula_str, data=transf_dtf)
results = lm1.fit()

In [ ]:
print(results.summary())

Most of the independent variables in this data set are not significant to the linear regression model. As seen before, only rd_spend has a clear linear relationship to profit.
We may need to find better data for building and thinkering with such kind of regression.